四个类别的数据
0 正常
1 模糊
2 透明遮挡
3 完全遮挡

In [47]:
from torch.utils.data import DataLoader, Dataset, random_split
import os
from PIL import Image
import torch
import random
# dataSet 处理
class ImgDataSet(Dataset):
    # 类构造器
    def __init__(self,main_folder,transform = None):
        self.main_folder = main_folder
        self.transform = transform
        self.images = []
        self.labels = []
        # pass

        # 遍历所有的文件夹，创建数据集
        for label in os.listdir(main_folder):
            label_folder = os.path.join(main_folder,label)  # 不同label图片的文件夹路径

            if os.path.isdir(label_folder):
                for image in os.listdir(label_folder):
                    image_path = os.path.join(label_folder,image)
                    self.images.append(image_path)
                    self.labels.append(int(label))


    # 获取到一个包含数据和标签的元组
    # 同时会动态加载图片，所以只需要存储图片路径即可
    def __getitem__(self, index):

        img_path = self.images[index]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        label = self.labels[index]  # 获取标签索引
        label = torch.tensor(label, dtype=torch.long)  # 转换为长整型 Tensor
        return image, label  # 确保返回的是 (image, label)
    # 获取数据集长度
    def __len__(self):
        return self.images.__len__()
        # pass

    def split_dataset(self, train_ratio=0.7, val_ratio=0.15):
        # 计算测试集比例
        test_ratio = 1 - train_ratio - val_ratio

        # 随机打乱数据
        indices = list(range(len(self.images)))
        random.shuffle(indices)

        train_size = int(len(self.images) * train_ratio)
        val_size = int(len(self.images) * val_ratio)

        train_indices = indices[:train_size]
        val_indices = indices[train_size:train_size + val_size]
        test_indices = indices[train_size + val_size:]

        return (
            ImgDataSetSubset(self, train_indices),
            ImgDataSetSubset(self, val_indices),
            ImgDataSetSubset(self, test_indices),
        )

class ImgDataSetSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, index):
        return self.dataset[self.indices[index]]

    def __len__(self):
        return len(self.indices)

In [48]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

main_folder = 'data/img'

dataset = ImgDataSet(main_folder=main_folder,transform=transform)

print(len(dataset))

41316


In [49]:
print(dataset[0])

(tensor([[[0.9765, 0.9373, 0.9882,  ..., 0.9686, 0.9843, 0.9961],
         [0.8078, 0.5765, 0.9294,  ..., 0.7961, 0.9490, 0.9961],
         [0.7647, 0.6314, 0.7882,  ..., 0.7647, 0.9451, 0.9922],
         ...,
         [0.0667, 0.6157, 0.7569,  ..., 0.2353, 0.2039, 0.0667],
         [0.0196, 0.2039, 0.3765,  ..., 0.0118, 0.0157, 0.0353],
         [0.0039, 0.0039, 0.0078,  ..., 0.0000, 0.0000, 0.0314]],

        [[0.9608, 0.9255, 0.9882,  ..., 0.9686, 0.9843, 0.9961],
         [0.7765, 0.5412, 0.9216,  ..., 0.7961, 0.9490, 0.9961],
         [0.7255, 0.5882, 0.7765,  ..., 0.7686, 0.9451, 0.9961],
         ...,
         [0.0745, 0.6196, 0.7569,  ..., 0.2353, 0.2039, 0.0745],
         [0.0235, 0.2039, 0.3765,  ..., 0.0157, 0.0196, 0.0392],
         [0.0078, 0.0078, 0.0039,  ..., 0.0157, 0.0118, 0.0431]],

        [[0.9608, 0.9294, 0.9922,  ..., 0.9725, 0.9843, 0.9961],
         [0.7569, 0.6039, 0.9373,  ..., 0.8000, 0.9529, 1.0000],
         [0.7020, 0.6784, 0.8000,  ..., 0.7725, 0.9490, 1

In [50]:
train_dataset,val_dataset,test_dataset = dataset.split_dataset()
print(len(train_dataset), len(val_dataset), len(test_dataset))

28921 6197 6198


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 定义 CNN 模型
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # 假设输入图像大小为224x224
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)  # 展平
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# 假设 dataset 是你已经创建好的 ImgDataSet 实例
train_dataset, val_dataset, test_dataset = dataset.split_dataset()
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 模型、损失函数和优化器
model = SimpleCNN(num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        # labels = labels.type(torch.long)  # 确保标签是长整型 Tensor
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 验证模型
model.eval()
with torch.no_grad():
    total, correct = 0, 0
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')


KeyboardInterrupt: 

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 定义 CNN 模型
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # 假设输入图像大小为224x224
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)  # 展平
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# 假设 dataset 是你已经创建好的 ImgDataSet 实例
train_dataset, val_dataset, test_dataset = dataset.split_dataset()
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 检查是否有可用的 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# 模型、损失函数和优化器
model = SimpleCNN(num_classes=4).to(device)  # 将模型转移到 GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # 将数据转移到 GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 验证模型
model.eval()
with torch.no_grad():
    total, correct = 0, 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # 将数据转移到 GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')


cuda


KeyboardInterrupt: 